# Correlations after clustering ASVs into OTUs

Given the difficulty of running correlations using very sparse ASV table generated with dada2 (min quality 20, only forward reads), we decided to try the clustering approach to group ASVs into OTUs. This comprises a higher level of resolution that could potentially work.

Qiime2 has nice tutorial on how to do this.

## Correlations with OTUs at 99% using SILVA (closed reference)


In [2]:
kremling_expression_key = '/media/renato/SSD1TB/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/0_kremling_expression_key.txt'
sra_run_table_16s = '/media/renato/SSD1TB/Repositories/maize_microbiome_transcriptomics/16S_wallace2018/SraRunInfo_Wallace_etal_2018.csv'
sra_run_table_rnaseq = '/media/renato/SSD1TB/Repositories/maize_microbiome_transcriptomics/rnaseq_kremling2018/run_info/SraRunInfo_Kremling_etal_2018.csv'

dict_wallace_kremling_2018 = {}
kremling_expression_key_dict = {}

In [3]:
with open(kremling_expression_key, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split('\t')
        
        kremling_identifier = fields[0]
        wallace_identifier = fields[1]

        kremling_expression_key_dict[kremling_identifier] = wallace_identifier

In [4]:
import re

with open(sra_run_table_rnaseq, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split(',')
        fields2 = fields[11].split('_')
        rnaseq_run_id = fields[0]
        sample_id = fields2[1]
        rnaseq_genotype = fields2[2]
        day = ''
        match = re.search(r'\d+', sample_id)
        unmatched_parts = re.split(r'\d+', sample_id)
        day_period = unmatched_parts[0]
        if match:
            day = int(match.group())
        if sample_id.startswith('LMA') and rnaseq_genotype != '#N/A':
            dict_wallace_kremling_2018[fields[11]] = {'run_accession_16s': '',
                                    'run_accession_rnaseq': rnaseq_run_id,
                                    'day': day,
                                    'day_period': day_period,
                                    'genotype_16s': '',
                                    'genotype_rnaseq': rnaseq_genotype}

In [5]:
rnaseq_samples_with_16s = 0

with open(sra_run_table_16s, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split(',')
        fields2 = fields[11].split('.')
        metataxonomics_run_id = fields[0]
        day = int(fields2[1])
        day_period = fields2[0]
        for key, value in kremling_expression_key_dict.items():
            if value == fields[11]:
                if dict_wallace_kremling_2018[key]['day'] != day:
                    print('Big problem!')
                    print(day, dict_wallace_kremling_2018[key]['day'])
                    print(dict_wallace_kremling_2018[key])
                    print(value, fields[11], key)
                    exit(1)
                if dict_wallace_kremling_2018[key]['day_period'] != day_period:
                    print('Big problem!')
                    if key == '10343927_LMAN8_CML505_CAACAG':
                        #print("It's ok. I know this sample is problematic.")
                        continue
                    else:
                        print(day_period, dict_wallace_kremling_2018[key]['day_period'])
                        print(dict_wallace_kremling_2018[key])
                        print(value, fields[11], key)
                        exit(1)
                dict_wallace_kremling_2018[key]['run_accession_16s'] = metataxonomics_run_id
                rnaseq_samples_with_16s+=1

print(f'{rnaseq_samples_with_16s} sample pairs found.')

Big problem!
484 sample pairs found.


In [6]:
no_16s = 0
for key, value in dict_wallace_kremling_2018.items():
    if value['run_accession_16s'] == '':
        print(key, value)
        no_16s+=1
print(f'{no_16s} samples without 16S data.')

10343927_LMAN8_B73_CACACT {'run_accession_16s': '', 'run_accession_rnaseq': 'SRR5909633', 'day': 8, 'day_period': 'LMAN', 'genotype_16s': '', 'genotype_rnaseq': 'B73'}
10343927_LMAN8_CML505_CAACAG {'run_accession_16s': '', 'run_accession_rnaseq': 'SRR5911345', 'day': 8, 'day_period': 'LMAN', 'genotype_16s': '', 'genotype_rnaseq': 'CML505'}
2 samples without 16S data.


In [7]:
run2my_sample_id = {}

for key in dict_wallace_kremling_2018:
    if dict_wallace_kremling_2018[key]['run_accession_rnaseq']:
        run2my_sample_id[dict_wallace_kremling_2018[key]['run_accession_rnaseq']] = key
    if dict_wallace_kremling_2018[key]['run_accession_16s']:
        run2my_sample_id[dict_wallace_kremling_2018[key]['run_accession_16s']] = key

In [8]:
import pandas as pd

# Importing expression data from Kremling et al. 2018 (TPM matrix on Maize v5 using Salmon after cleaning with cutadapt)
#kremling_expression_v5 = pd.read_csv('/media/rsantos/4TB_drive/Projects/UGA_RACS/RNAseq/Salmon/Zma2_tpm_matrix.txt', sep='\t')
kremling_expression_v5 = pd.read_csv('/media/renato/SSD1TB/Projects/UGA_RACS/RNAseq/Salmon/Zma2_tpm_matrix.txt', sep='\t')

# Rename column and reset the index
kremling_expression_v5.set_index('Name', inplace=True)

# Print the dataframe
kremling_expression_v5.head()

,SRR5909626,SRR5909627,SRR5909633,SRR5909635,SRR5909639,SRR5909642,SRR5909645,SRR5909653,SRR5909655,SRR5909665,...,SRR5912073,SRR5912081,SRR5912082,SRR5912083,SRR5912093,SRR5912094,SRR5912104,SRR5912105,SRR5912111,SRR5912116
Name,,,,,,,,,,,,,,,,,,,,,
Zm00001eb371370_T002,1.04145,0.0000,3.391060,0.0,0.0000,1.82712,0.284514,2.23201,0.437147,0.468934,...,0.00000,1.51042,0.00000,0.0,0.00000,2.82055,3.96967,0.00000,2.96105,0.00000
Zm00001eb371350_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371330_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371310_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371280_T001,1.27650,2.1092,0.692731,0.0,4.2798,1.47496,2.557320,0.00000,1.065940,1.149530,...,3.02253,0.41140,1.17447,0.0,3.48749,9.47506,6.19189,3.80776,1.03695,1.14981


In [10]:
# Rename the columns using the dictionary
kremling_expression_v5 = kremling_expression_v5.rename(columns=run2my_sample_id)
kremling_expression_v5.columns = [str(x) for x in kremling_expression_v5.columns]

kremling_expression_v5.head()

,10343927_LMAD26_CI21E_AAGTGG,10343264_LMAN26_CI21E_ATGAAC,10343927_LMAN8_B73_CACACT,10343264_LMAN26_B64_ACCAGT,10343262_LMAN8_B109_TGCTAT,10343262_LMAN8_B14A_CTCTCG,10343262_LMAN8_B57_CCTAAG,10343927_LMAD26_B77_TAATCG,10343262_LMAN8_B79_GCAGCC,10343927_LMAN8_CI187-2_GACGAT,...,10344826_LMAN8_I29_ACGTCT,10344823_LMAD8_IA2132_ACACGC,10343264_LMAD26_CML91_AACGCC,10344827_LMAN26_CML91_AATCCG,10344827_LMAN26_Ki21_AAGACA,10343927_LMAD26_Ki21_ACGTCT,10344826_LMAD8_E2558W_CGCAAC,10343927_LMAN8_E2558W_GAACCT,10344826_LMAD8_IDS69_CAGGAC,10343927_LMAN8_IDS69_ACATTA
Name,,,,,,,,,,,,,,,,,,,,,
Zm00001eb371370_T002,1.04145,0.0000,3.391060,0.0,0.0000,1.82712,0.284514,2.23201,0.437147,0.468934,...,0.00000,1.51042,0.00000,0.0,0.00000,2.82055,3.96967,0.00000,2.96105,0.00000
Zm00001eb371350_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371330_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371310_T001,0.00000,0.0000,0.000000,0.0,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Zm00001eb371280_T001,1.27650,2.1092,0.692731,0.0,4.2798,1.47496,2.557320,0.00000,1.065940,1.149530,...,3.02253,0.41140,1.17447,0.0,3.48749,9.47506,6.19189,3.80776,1.03695,1.14981


### Importing feature table with ASV-generated OTU at 99% (SILVA)

ASVs generated from dada2 with reads quality 20, only forward reads, were clustered into OTUs at 99% using SILVA as reference (pre-processing steps are described on GitLab).

Here, I (RACS) import this table:

In [18]:
wallace_asvs_q20_fw_closedref_silva = pd.read_csv('/media/renato/SSD1TB/Projects/UGA_RACS/16S/Qiime2/clustering/q20_forward/close_ref_99/cr-wallace2018_forward_q20-99_feature-table/cr99_q20_rv_feature-table.tsv',
                           sep='\t', comment='#')

# Rename column and reset the index
wallace_asvs_q20_fw_closedref_silva.rename(columns={'OTU ID': 'Name'}, inplace=True)
wallace_asvs_q20_fw_closedref_silva.set_index('Name', inplace=True)

# Print the dataframe
wallace_asvs_q20_fw_closedref_silva.head(n=2)

,SRR6665476,SRR6665477,SRR6665478,SRR6665479,SRR6665480,SRR6665481,SRR6665482,SRR6665483,SRR6665484,SRR6665485,...,SRR6666058,SRR6666059,SRR6666060,SRR6666061,SRR6666062,SRR6666063,SRR6666064,SRR6666065,SRR6666066,SRR6666067
Name,,,,,,,,,,,,,,,,,,,,,
HG328252.1.1392,47869.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,78028.0,0.0,0.0,0.0,0.0,0.0,58946.0,3868.0,0.0
AAAA02020714.1.1202,0.0,4494.0,7158.0,47516.0,12429.0,5135.0,4201.0,1389.0,3836.0,1433.0,...,30209.0,0.0,16345.0,23309.0,55388.0,2754.0,19631.0,0.0,0.0,5564.0


In [19]:
# Rename the columns using the dictionary
wallace_asvs_q20_fw_closedref_silva = wallace_asvs_q20_fw_closedref_silva.rename(columns=run2my_sample_id)
wallace_asvs_q20_fw_closedref_silva.columns = [str(x) for x in wallace_asvs_q20_fw_closedref_silva.columns]

In [20]:
wallace_asvs_q20_fw_closedref_silva.head()

,SRR6665476,10343264_LMAN26_B73_GTGTAG,10343264_LMAN26_NC262_ACAGAT,10343264_LMAN26_CML10_AGACCA,10343264_LMAN26_NC314_ACGTCT,10343264_LMAN26_B46_ACCGTG,10343264_LMAN26_B84_GTGCCA,10343264_LMAN26_B73_ACTCTT,10343264_LMAN26_B77_GTAGAA,10344826_LMAN8_F7_GGCTGC,...,SRR6666058,SRR6666059,SRR6666060,SRR6666061,10344827_LMAN26_I137TN_ACATTA,10343264_LMAN26_CI64_ATATCC,10343927_LMAD26_CML154Q_ACAGAT,10343927_LMAD26_T234_GTCAGG,10343927_LMAD26_NC344_AATGAA,10343927_LMAD26_K64_CCTGCT
Name,,,,,,,,,,,,,,,,,,,,,
HG328252.1.1392,47869.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,78028.0,0.0,0.0,0.0,0.0,0.0,58946.0,3868.0,0.0
AAAA02020714.1.1202,0.0,4494.0,7158.0,47516.0,12429.0,5135.0,4201.0,1389.0,3836.0,1433.0,...,30209.0,0.0,16345.0,23309.0,55388.0,2754.0,19631.0,0.0,0.0,5564.0
KF101313.1.1306,0.0,1000.0,6839.0,478.0,11097.0,12190.0,11492.0,12074.0,19987.0,2239.0,...,1253.0,0.0,22029.0,367.0,14655.0,75048.0,2044.0,0.0,0.0,1542.0
FN421747.1.1368,0.0,2086.0,7930.0,13604.0,19280.0,8901.0,1051.0,667.0,752.0,14335.0,...,237.0,0.0,14597.0,4431.0,12959.0,478.0,9602.0,0.0,0.0,227.0
MH337967.1.1252,0.0,1351.0,5537.0,10010.0,13543.0,6101.0,766.0,407.0,446.0,9845.0,...,152.0,0.0,8222.0,2908.0,10874.0,293.0,6294.0,0.0,0.0,193.0


In [21]:
kremling_expression_v5 = kremling_expression_v5.filter(items=wallace_asvs_q20_fw_closedref_silva.columns)

In [22]:
wallace_asvs_q20_fw_closedref_silva = wallace_asvs_q20_fw_closedref_silva.filter(items=kremling_expression_v5.columns)

In [23]:
print(kremling_expression_v5.shape)
kremling_expression_v5.head()

(39096, 482)


,10343264_LMAN26_B73_GTGTAG,10343264_LMAN26_NC262_ACAGAT,10343264_LMAN26_CML10_AGACCA,10343264_LMAN26_NC314_ACGTCT,10343264_LMAN26_B46_ACCGTG,10343264_LMAN26_B84_GTGCCA,10343264_LMAN26_B73_ACTCTT,10343264_LMAN26_B77_GTAGAA,10344826_LMAN8_F7_GGCTGC,10344826_LMAN8_ND246_CGTCGC,...,10344826_LMAD8_NC358_GCAGCC,10344826_LMAD8_NC294_CGATCT,10344826_LMAD8_K55_AAGACA,10344827_LMAN26_B73_GAACCT,10344827_LMAN26_I137TN_ACATTA,10343264_LMAN26_CI64_ATATCC,10343927_LMAD26_CML154Q_ACAGAT,10343927_LMAD26_T234_GTCAGG,10343927_LMAD26_NC344_AATGAA,10343927_LMAD26_K64_CCTGCT
Name,,,,,,,,,,,,,,,,,,,,,
Zm00001eb371370_T002,1.349820,0.0,0.000000,0.0,0.000000,0.359664,0.541724,0.000000,0.00000,0.0,...,2.11575,1.24155,1.24528,0.00000,0.0,0.00000,0.808996,0.743690,3.42920,0.00000
Zm00001eb371350_T001,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.000000,0.000000,0.00000,0.00000
Zm00001eb371330_T001,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.000000,0.000000,0.00000,0.00000
Zm00001eb371310_T001,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.000000,0.000000,0.00000,0.00000
Zm00001eb371280_T001,0.441188,0.0,0.215403,0.0,0.817563,2.351130,0.885315,0.517418,1.56597,0.0,...,2.99848,2.70535,2.00021,2.23636,0.0,2.35318,2.974720,0.911501,1.52984,4.15375


In [24]:
print(wallace_asvs_q20_fw_closedref_silva.shape)
wallace_asvs_q20_fw_closedref_silva.head()

(1896, 482)


,10343264_LMAN26_B73_GTGTAG,10343264_LMAN26_NC262_ACAGAT,10343264_LMAN26_CML10_AGACCA,10343264_LMAN26_NC314_ACGTCT,10343264_LMAN26_B46_ACCGTG,10343264_LMAN26_B84_GTGCCA,10343264_LMAN26_B73_ACTCTT,10343264_LMAN26_B77_GTAGAA,10344826_LMAN8_F7_GGCTGC,10344826_LMAN8_ND246_CGTCGC,...,10344826_LMAD8_NC358_GCAGCC,10344826_LMAD8_NC294_CGATCT,10344826_LMAD8_K55_AAGACA,10344827_LMAN26_B73_GAACCT,10344827_LMAN26_I137TN_ACATTA,10343264_LMAN26_CI64_ATATCC,10343927_LMAD26_CML154Q_ACAGAT,10343927_LMAD26_T234_GTCAGG,10343927_LMAD26_NC344_AATGAA,10343927_LMAD26_K64_CCTGCT
Name,,,,,,,,,,,,,,,,,,,,,
HG328252.1.1392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,156104.0,91111.0,90631.0,0.0,0.0,0.0,0.0,58946.0,3868.0,0.0
AAAA02020714.1.1202,4494.0,7158.0,47516.0,12429.0,5135.0,4201.0,1389.0,3836.0,1433.0,229.0,...,0.0,0.0,0.0,731.0,55388.0,2754.0,19631.0,0.0,0.0,5564.0
KF101313.1.1306,1000.0,6839.0,478.0,11097.0,12190.0,11492.0,12074.0,19987.0,2239.0,519.0,...,0.0,0.0,0.0,2957.0,14655.0,75048.0,2044.0,0.0,0.0,1542.0
FN421747.1.1368,2086.0,7930.0,13604.0,19280.0,8901.0,1051.0,667.0,752.0,14335.0,285.0,...,0.0,0.0,0.0,2362.0,12959.0,478.0,9602.0,0.0,0.0,227.0
MH337967.1.1252,1351.0,5537.0,10010.0,13543.0,6101.0,766.0,407.0,446.0,9845.0,200.0,...,0.0,0.0,0.0,1775.0,10874.0,293.0,6294.0,0.0,0.0,193.0


In [26]:
if wallace_asvs_q20_fw_closedref_silva.columns.all() == kremling_expression_v5.columns.all():
    print('Columns are equal!')

Columns are equal!


In [27]:
def count_zeros(df, threshold=0.5):
    # Count the number of zeros in each row
    # If the number of zeros is greater than the threshold, remove the row
    threshold_int = int(df.shape[1] * threshold)
    print(f'Threshold: {threshold_int} (threshold * number of columns)')
    zero_counts = df.apply(lambda row: (row == 0).sum(), axis=1)
    return df[zero_counts < threshold_int]

In [28]:
wallace_asvs_q20_fw_closedref_silva_zeros_filtered = count_zeros(wallace_asvs_q20_fw_closedref_silva)
kremling_expression_v5_zeros_filtered = count_zeros(kremling_expression_v5)

Threshold: 241 (threshold * number of columns)
Threshold: 241 (threshold * number of columns)


In [29]:
print(wallace_asvs_q20_fw_closedref_silva_zeros_filtered.shape)
print(kremling_expression_v5_zeros_filtered.shape)

(7, 482)
(19953, 482)


In [58]:
wallace_asvs_q20_fw_closedref_silva_zeros_filtered

,10343264_LMAN26_B73_GTGTAG,10343264_LMAN26_NC262_ACAGAT,10343264_LMAN26_CML10_AGACCA,10343264_LMAN26_NC314_ACGTCT,10343264_LMAN26_B46_ACCGTG,10343264_LMAN26_B84_GTGCCA,10343264_LMAN26_B73_ACTCTT,10343264_LMAN26_B77_GTAGAA,10344826_LMAN8_F7_GGCTGC,10344826_LMAN8_ND246_CGTCGC,...,10344826_LMAD8_NC358_GCAGCC,10344826_LMAD8_NC294_CGATCT,10344826_LMAD8_K55_AAGACA,10344827_LMAN26_B73_GAACCT,10344827_LMAN26_I137TN_ACATTA,10343264_LMAN26_CI64_ATATCC,10343927_LMAD26_CML154Q_ACAGAT,10343927_LMAD26_T234_GTCAGG,10343927_LMAD26_NC344_AATGAA,10343927_LMAD26_K64_CCTGCT
Name,,,,,,,,,,,,,,,,,,,,,
AAAA02020714.1.1202,4494.0,7158.0,47516.0,12429.0,5135.0,4201.0,1389.0,3836.0,1433.0,229.0,...,0.0,0.0,0.0,731.0,55388.0,2754.0,19631.0,0.0,0.0,5564.0
KF101313.1.1306,1000.0,6839.0,478.0,11097.0,12190.0,11492.0,12074.0,19987.0,2239.0,519.0,...,0.0,0.0,0.0,2957.0,14655.0,75048.0,2044.0,0.0,0.0,1542.0
FN421747.1.1368,2086.0,7930.0,13604.0,19280.0,8901.0,1051.0,667.0,752.0,14335.0,285.0,...,0.0,0.0,0.0,2362.0,12959.0,478.0,9602.0,0.0,0.0,227.0
MH337967.1.1252,1351.0,5537.0,10010.0,13543.0,6101.0,766.0,407.0,446.0,9845.0,200.0,...,0.0,0.0,0.0,1775.0,10874.0,293.0,6294.0,0.0,0.0,193.0
JF177515.1.1306,636.0,7046.0,652.0,7960.0,85873.0,901.0,652.0,5983.0,7041.0,227.0,...,0.0,0.0,0.0,197.0,3584.0,4680.0,190.0,0.0,0.0,303.0
JQ904048.1.1214,286.0,26752.0,288.0,31585.0,7985.0,1031.0,424.0,6752.0,12079.0,534.0,...,0.0,0.0,0.0,308.0,3406.0,961.0,118.0,0.0,0.0,619.0
HM487994.1.1230,184.0,10256.0,152.0,8088.0,6904.0,903.0,123.0,452.0,2735.0,0.0,...,0.0,0.0,0.0,526.0,1881.0,565.0,1773.0,0.0,0.0,185.0


In [57]:
test_file = open('/media/renato/SSD1TB/Projects/UGA_RACS/16S/Qiime2/clustering/q20_forward/close_ref_99/cr-wallace2018_forward_q20-99_feature-table/cr99_q20_rv_feature-table.tsv')

more_than_241 = 0

with test_file as file:
    for line in file:
        if line.startswith('#'):
            continue
        if line.startswith('OTU ID'):
            continue
        fields = line.split('\t')
        fields.pop(0)
        #print(fields)
        fields_int = [round(float(i)) for i in fields]
        count_zeros = fields_int.count(0)
        if count_zeros < 280:
            print(print(fields_int))
            print(count_zeros)       


[0, 4494, 7158, 47516, 12429, 5135, 4201, 1389, 3836, 1433, 229, 0, 5782, 0, 15708, 31639, 2227, 0, 12458, 15150, 8789, 0, 0, 0, 0, 0, 0, 938, 0, 0, 2974, 0, 0, 0, 2154, 0, 0, 0, 0, 64891, 20525, 103466, 15562, 0, 1669, 0, 8977, 0, 0, 0, 0, 0, 0, 3975, 0, 648, 0, 16679, 0, 0, 757, 0, 267, 1201, 0, 0, 0, 0, 38752, 0, 44671, 0, 0, 23320, 0, 0, 11477, 0, 1791, 0, 1573, 0, 1197, 8220, 0, 0, 200, 10808, 0, 16164, 152, 0, 0, 15953, 81452, 1518, 0, 456, 4168, 0, 107579, 1830, 398, 13591, 13330, 44869, 0, 22193, 23339, 0, 0, 0, 0, 0, 8274, 0, 0, 53210, 4176, 0, 321, 262, 0, 0, 0, 1079, 297, 0, 0, 0, 13581, 75175, 0, 587, 1836, 0, 7103, 0, 7299, 0, 0, 14636, 0, 5867, 0, 7592, 0, 3063, 639, 10384, 67017, 0, 3963, 3063, 1220, 3203, 1387, 2096, 4327, 656, 0, 188358, 1088, 7157, 0, 6286, 0, 0, 0, 4249, 0, 0, 290, 36162, 0, 22459, 61590, 10955, 0, 7034, 0, 6031, 0, 0, 53300, 10507, 4889, 0, 0, 0, 114995, 20405, 9377, 25741, 25224, 0, 42794, 68444, 1600, 0, 33964, 4482, 5447, 5157, 0, 0, 5636, 0, 290

## Correlations with OTUs at 97% (de novo clustering)